In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:  
        print(os.path.join(dirname, filename))  

创建绘图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json


# 训练日志绘图类
class HistoryGraph:
    def __init__(self, history, epochs, title, file_path):
        # 训练日志
        self.history = history
        # 训练趟数
        self.epochs = epochs
        # 图标题
        self.title = title
        # 图像存盘文件路径名
        self.file_path = file_path

    def draw(self):
        figure, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        figure.suptitle(self.title, fontsize=12)
        figure.subplots_adjust(top=0.85, wspace=0.3)

        epoch_list = list(range(1, self.epochs + 1))
        ax1.plot(epoch_list,
                 self.history.history['accuracy'],
                 label='Train Accuracy')
        ax1.plot(epoch_list,
                 self.history.history['val_accuracy'],
                 label='Validation Accuracy')
        ax1.set_xticks(np.arange(0, self.epochs + 1, 5))
        ax1.set_ylabel('Accuracy Value')
        ax1.set_xlabel('Epoch #')
        ax1.set_title('Accuracy')
        ax1.legend(loc="best")

        ax2.plot(epoch_list, self.history.history['loss'], label='Train Loss')
        ax2.plot(epoch_list,
                 self.history.history['val_loss'],
                 label='Validation Loss')
        ax2.set_xticks(np.arange(0, self.epochs + 1, 5))
        ax2.set_ylabel('Loss Value')
        ax2.set_xlabel('Epoch #')
        ax2.set_title('Loss')
        ax2.legend(loc="best")
        figure.savefig(self.file_path)
        figure.show()
        plt.close()

    def draw_from_json(self, json_path):
        history = json.loads(open(json_path).read())
        epochs = len(history["loss"])
        figure, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        figure.suptitle(self.title, fontsize=12)
        figure.subplots_adjust(top=0.85, wspace=0.3)

        epoch_list = list(range(1, epochs + 1))
        ax1.plot(epoch_list,
                 history['accuracy'],
                 label='Train Accuracy')
        ax1.plot(epoch_list,
                 history['val_accuracy'],
                 label='Validation Accuracy')
        ax1.set_xticks(np.arange(0, epochs + 1, 5))
        ax1.set_ylabel('Accuracy Value')
        ax1.set_xlabel('Epoch #')
        ax1.set_title('Accuracy')
        ax1.legend(loc="best")

        ax2.plot(epoch_list, history['loss'], label='Train Loss')
        ax2.plot(epoch_list,
                 history['val_loss'],
                 label='Validation Loss')
        ax2.set_xticks(np.arange(0, epochs + 1, 5))
        ax2.set_ylabel('Loss Value')
        ax2.set_xlabel('Epoch #')
        ax2.set_title('Loss')
        ax2.legend(loc="best")
        figure.savefig(self.file_path)
        plt.close()

处理csv文件

In [ ]:
import pandas as pd
sample_df = pd.read_csv('/kaggle/input/facial-expression-recognition-challenge/icml_face_data.csv/icml_face_data.csv')
print(sample_df)
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 修改列名称：emotion→label，pixels→image
sample_df.columns = ['label', 'Usage', 'image']

# 删除Usage列
sample_df = sample_df.drop(columns=['Usage'])
print(sample_df)

# 添加emotion列及对应的内容
sample_df['emotion'] = sample_df['label'].apply(lambda x: emotions[int(x)])
print(sample_df)

统计样本分布

In [ ]:
sample_distribution_df = sample_df.groupby('emotion').count()
print(sample_distribution_df)

sample_df = sample_df[sample_df['emotion']!='Disgust']
sample_distribution_df = sample_df.groupby('emotion').count()
print(sample_distribution_df)

采样前后数量

In [ ]:
m = sample_df.groupby('label').count().mean().values[0]
print("各类表情样本平均数量: " + str(m))

emotions = ['Angry', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

oversampled = pd.DataFrame()
for emotion in emotions:
    print('\n' + emotion)
    l = len(sample_df[sample_df.emotion==emotion])
    print('采样前: ' + str(l))
    
    if (l>=m):
        dft = sample_df[sample_df.emotion==emotion].sample(int(m))
        oversampled = oversampled.append(dft)
        print('采样后: ' + str(len(dft)))
    else:
        frac = int(m/l)
        dft = pd.DataFrame()
        for i in range(frac+1):
            dft = dft.append(sample_df[sample_df.emotion==emotion])
        dft = dft[dft.emotion==emotion].sample(int(m))
        oversampled = oversampled.append(dft)
        print('采样后: ' + str(len(dft)))
        
oversampled = oversampled.sample(frac=1).reset_index().drop(columns=['index'])
print(oversampled)
sample_distribution_df = oversampled.groupby('emotion').count()
print(sample_distribution_df)

分割训练集、测试集

In [ ]:
from sklearn.model_selection import train_test_split

# 删除emotion列
# oversampled = oversampled.drop(columns=['emotion'])

# 随机混洗DataFrame的行 
oversampled = oversampled.sample(frac = 1) 

train_sample, test_sample = train_test_split(oversampled, test_size=0.1)
print(train_sample)
print(test_sample)

MiniVGG11模型

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras import backend


# 定义 MiniVGG11类
class MiniVGG11:
    @staticmethod
    def build(width, height, channel, classes, reg=0.0002):
        """
        根据输入样本的维度（width、height、channel），分类数量，正则化因子创建MiniVGG11网络模型
        Args:
            width:   输入样本的宽度
            height:  输入样本的高度
            channel: 输入样本的通道
            classes: 分类数量
            reg:     正则化因子

        Returns:
            MiniVGG11网络模型对象

        """

        model = Sequential(name="MiniVGG11")

        # 缺省输入格式为通道后置（"channels_last"）
        shape = (height, width, channel)
        channel_dimension = -1

        # 如果输入格式为通道前置
        # 重新设置输入格式和通道位置指示
        if backend.image_data_format() == "channels_first":
            shape = (channel, height, width)
            channel_dimension = 1

        # 第一卷积块
        model.add(Conv2D(64, (3, 3), input_shape=shape, padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第二卷积块
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第三卷积块
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))

        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第四卷积块
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第五卷积块
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same", strides=(1, 1)))

        # 第一全连接层
        model.add(Flatten())
        model.add(Dense(32, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        
        # 第二全连接层
        model.add(Dense(32, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        # 第三全连接层
        model.add(Dense(classes, kernel_regularizer=l2(reg)))
        model.add(Activation("softmax"))

        return model


# 测试MiniVGG11类实例化并输出MiniVGG11模型的概要信息
if __name__ == "__main__":
    model = MiniVGG11.build(width=48, height=48, channel=1, classes=7, reg=0.0002)
    print(model.summary())

更新学习率

In [ ]:
import numpy as np


class StepDecay:
    def __init__(self, init_alpha=0.256, factor=0.97, drop_every=2.4):
        # 保存初始学习速率、递减因子、递减周期
        self.initAlpha = init_alpha
        self.factor = factor
        self.dropEvery = drop_every

    def __call__(self, epoch):
        # 为当前训练趟就是新的学习速率
        exp = np.floor((1 + epoch) / self.dropEvery)
        alpha = self.initAlpha * (self.factor ** exp)
        # 返回新的学习速率
        return float(alpha)

模型训练

In [ ]:
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import numpy as np
from tensorflow.keras.callbacks import LearningRateScheduler


EPOCHS = 100
BATCH_SIZE = 512
NUM_CLASSES = 7
MODEL_FILE = "MiniVGG11_model.h5"

reg = 1e-4
learning_rate = 5e-4
opt = Nadam(learning_rate=learning_rate)

schedule = StepDecay(init_alpha=learning_rate, factor=0.5, drop_every=10)

lrs = LearningRateScheduler(schedule, verbose=1)

model = MiniVGG11.build(width=48, height=48, channel=1, classes=NUM_CLASSES, reg=reg)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# 训练网络
images = train_sample['image'].tolist()
X = []
for image in images:
    image = np.array(image.split(" "), dtype="float32")
    # image = image / 255.0
    image = image.reshape((48, 48, 1))
    X.append(image)

X = np.array(X)


labels = train_sample.loc[:,'label']
Y = to_categorical(labels)
history = model.fit(x=X,
                    y=Y, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    shuffle=True,
                    validation_split=0.2, 
                    validation_batch_size=BATCH_SIZE, 
                    callbacks=[lrs,])

# 将训练得到的模型保存到文件
print("[信息] 保存模型...")
model.save(MODEL_FILE, overwrite=True)

# 绘制并保存训练性能图
title = "VGG11 Fer2013 Dataset Training Performance 320203313"
graph_file = "VGG11_Fer2013_Dataset_Training_Performance.pdf"
HistoryGraph(history, EPOCHS, title, graph_file).draw()

# 保存训练历史数据
json_path = "VGG11_Fer2013_Dataset_Training.json"
f = open(json_path, "w")
f.write(json.dumps(str(history.history), skipkeys=True))
f.close()

模型评估

In [ ]:
from keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import load_model
# 训练网络
images = test_sample['image'].tolist()
X = []
for image in images:
    image = np.array(image.split(" "), dtype="float32")
    # image = image / 255.0
    image = image.reshape((48, 48, 1))
    X.append(image)

X = np.array(X)


labels = test_sample.loc[:,'label']
Y = to_categorical(labels)

# 加载前面训练好的网络
print("[信息] 加载网络模型...")
model = load_model("../input/vgg11model/MiniVGG11_model.h5")

(loss, acc) = model.evaluate(x=X, y=Y)
print("[信息] 测试集准确率：{:.2f}%".format(acc * 100))